<a href="https://colab.research.google.com/github/khorzhengyu/project518/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Amount Variable ##
results = pd.concat(
    [
        df['amt'].describe(percentiles=[0.5, 0.95, 0.999]).reset_index().rename(columns={'index': 'Row Type', 'amt': 'Overall Amt Distribution'}).round(3),
        df.loc[df['isfraud'] == 0, ['amt']].describe(percentiles=[0.5, 0.95, 0.999]).reset_index(drop=1).rename(columns={'amt': 'Non-Fraud Amt Distribution'}).round(3),
        df.loc[df['isfraud'] == 1, ['amt']].describe(percentiles=[0.5, 0.95, 0.999]).reset_index(drop=1).rename(columns={'amt': 'Fraud Amt Distribution'}).round(3)
    ], axis=1)
results

In [ ]:
all_trans = df.copy()
all_trans['class'] = all_trans['isfraud'].map({1:'Fraud',0:'Non-Fraud'})
normal = all_trans[df['isfraud']==0]
fraud = all_trans[df['isfraud'] == 1]

def stats_by_class(variable):
  stat_grid = all_trans.groupby('class')[variable].agg([np.min,np.max,np.mean,np.median])
  stat_grid = stat_grid.transpose().round(2)
  return stat_grid

def plot_box(data, x,y,title,width = 10, height = 7):
  plt.figure(figsize = [width,height])
  sns.boxplot(data = data, x=x,y=y)
  plt.title(title)

#stats_by_class('amt') # calling the stats_by class function

plot_box(all_trans,'class','amt','Distribution of Amount vs Class'); # calling the boxplot function

fig = plt.subplots(figsize= (15,10))
plots = []
plots.append(sns.histplot(df[(df.isfraud == 0) & (df.amt <= 2000)].amt, bins=50, ax=plt.subplot(235)))
plots.append(sns.histplot(df[(df.isfraud ==1) & (df.amt <= 1500)].amt, bins =50, ax=plt.subplot(236)))

plots[0].set_title('Non-Fraud Amt Dist')
plots[1].set_title('Fraud Amt Dist')
plots[0].set_xlabel('Transaction Amount')
plots[1].set_xlabel('Transaction Amount')
plots[0].set_ylabel('Number of Transactions')
plots[1].set_ylabel('Number of Transactions')
plt.show()

stats_by_class('amt') # calling the stats_by class function

In [ ]:
ax = sns.histplot(x='amt', data=df[df.amt <= 2000], hue='isfraud',stat = 'percent',multiple= 'dodge',common_norm= False, bins = 25)
ax.set_ylabel('Percentage in Each Type')
ax.set_xlabel('Transaction amount in USD')
plt.legend(title='Type', labels = ['Fraud','Non-Fraud'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_ratio(df, bin_edges):
    # Create a new column 'amount_bin' based on transaction amount bins
    df['amount_bin'] = pd.cut(df['amt'], bins=bin_edges)

    # Group the DataFrame by 'amount_bin' and calculate counts of fraud and non-fraud transactions
    grouped = df.groupby('amount_bin')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the fraud-to-non-fraud ratio for each amount bin
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped


# Define the bin edges for transaction amounts
bin_edges = [0, 500, 1000, 2000]
bin_labels = ['0-499','500-999','1000-1999']
# Calculate the fraud-to-non-fraud ratio for transaction amounts
ratio_df = calculate_ratio(df, bin_edges)

# Convert the amount_bin intervals to strings
ratio_df['amount_bin'] = ratio_df['amount_bin'].astype(str)

# Sort the ratio_df DataFrame by fraud_ratio in ascending order
ratio_df_sorted = ratio_df.sort_values('fraud_ratio')

print (ratio_df_sorted)
# Plot all the groups on the bar chart in ascending order
plt.figure(figsize=(10, 5))
plt.bar(ratio_df_sorted['amount_bin'], ratio_df_sorted['fraud_ratio'])
plt.xlabel('Transaction Amount ($)')
plt.ylabel('Fraud Ratio')
plt.title('Fraud Ratio by Transaction Amount')
plt.xticks(ratio_df_sorted['amount_bin'],bin_labels, rotation=45)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df, bin_edges):
    # Create a new column 'amount_bin' based on transaction amount bins
    df['amount_bin'] = pd.cut(df['amt'], bins=bin_edges)

    # Group the DataFrame by 'amount_bin' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('amount_bin')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each amount bin
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    # Calculate the normalized non-fraud-to-fraud ratio for each amount bin
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

@ticker.FuncFormatter
def major_formatter(x, pos):
    label = str(-x) if x < 0 else str(x)
    return label

# Define the bin edges for transaction amounts
bin_edges = [0, 500, 1000, 2000]
bin_labels = ['0-499', '500-999', '1000-1999']
# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for transaction amounts
ratio_df = calculate_ratio(df, bin_edges)

# Convert the amount_bin intervals to strings
ratio_df['amount_bin'] = ratio_df['amount_bin'].astype(str)

# Sort the ratio_df DataFrame by fraud_ratio in ascending order
ratio_df_sorted = ratio_df.sort_values('fraud_ratio')

# Normalize the non-fraud ratio in the same way as the fraud ratio
max_non_fraud_ratio = ratio_df_sorted['fraud_ratio'].max()
ratio_df_sorted['non_fraud_ratio'] /= max_non_fraud_ratio

# Plot both normalized fraud and non-fraud ratios on the same bar chart
fig, ax1 = plt.subplots(figsize=(10, 5))

# Primary y-axis for Fraud Ratio
fraud_bars = ax1.barh(ratio_df_sorted['amount_bin'], ratio_df_sorted['fraud_ratio'], label='Fraud', color='blue')
ax1.set_xlabel('Fraud Ratio')
ax1.xaxis.set_major_formatter(major_formatter)
ax1.set_ylabel('Transaction Amount ($)')
ax1.set_yticks(ratio_df_sorted['amount_bin'])
ax1.set_yticklabels(bin_labels)


# Secondary y-axis for Non-Fraud Ratio
ax2 = ax1.twiny()
non_fraud_bars = ax2.barh(ratio_df_sorted['amount_bin'], -ratio_df_sorted['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
ax2.set_xlabel('Non-Fraud Ratio')
ax2.xaxis.tick_top()
ax2.xaxis.set_label_position('top')
ax2.xaxis.set_major_formatter(major_formatter)

# Display the ratios on the bars with adjusted positions and rotated text
for bar, fraud_ratio, non_fraud_ratio in zip(fraud_bars, ratio_df_sorted['fraud_ratio'], ratio_df_sorted['non_fraud_ratio']):
    ax1.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{fraud_ratio:.2f}', va='center', color='blue', fontsize=10, rotation=90)

for bar, non_fraud_ratio in zip(non_fraud_bars, ratio_df_sorted['non_fraud_ratio']):
    ax2.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{non_fraud_ratio:.2f}', va='center', ha='right', color='black', fontsize=10, rotation=90)

# Add legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='best')

plt.title('Fraud and Non-Fraud Ratios by Transaction Amount')
plt.tight_layout()
plt.show()

In [ ]:
## Age Variable ##
df['trans_hour']= df['transdatetranstime'].dt.hour
df['trans_day_of_week'] = df['transdatetranstime'].dt.day_name()
#df['trans_year_month'] = df['transdatetranstime'].dt.to_period('M')
df.head()
df.dtypes

In [ ]:
# calculate the age of customer
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = ((df['transdatetranstime'] - df['dob']) / np.timedelta64(1, 'Y')).round(1)
df.age

In [ ]:
# Bin the age feature using cut function
age_bins = [0, 30, 45, 60, 75, np.inf]
age_labels = ['< 30', '30-45', '46-60', '61-75', '> 75']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

df['age_group'].head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker  # Import ticker module for formatting

def calculate_ratio(df, bin_edges, column):
    # Create a new column 'group' based on binning the specified column
    df['group'] = pd.cut(df[column], bins=bin_edges)

    # Group the DataFrame by 'group' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('group')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each age group
    grouped['fraud_ratio'] = grouped[1] / grouped[0]
    # Calculate the normalized non-fraud-to-fraud ratio for each age group
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

# Define the bin edges for age groups
age_bin_edges = [0, 30, 45, 60, 75, np.inf]
bin_labels = ['< 30', '30 - 44', '45 - 59', '60 - 74', '75+']

# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for age groups
age_ratio_df = calculate_ratio(df, age_bin_edges, 'age')

# Convert the age group intervals to strings
age_ratio_df['age_group'] = age_ratio_df['group'].astype(str)

# Plot both normalized fraud and non-fraud ratios on the same bar chart
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(age_ratio_df['age_group'], age_ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(age_ratio_df['age_group'], -age_ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio

for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.xlabel('Ratio')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))  # Set x-axis formatter
plt.ylabel('Age Group')
plt.title('Fraud and Non-Fraud Ratios by Age Group')
plt.legend(loc='upper left')  # Adjust the legend position
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize = (10,5))
ax = sns.histplot(data = df, x = 'trans_hour', hue= 'isfraud', common_norm = False, stat = 'percent', multiple = 'dodge')
ax.set_xlabel('Time(Hour) in a Day')
ax.set_ylabel('Percentage')
plt.xticks(np.arange(0,24,1))
plt.legend(title = 'Type', labels = ['Fraud','Non-Fraud'])
plt.show()

In [ ]:
def map_time_of_day(hour):
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

# Apply the mapping function to create the 'time_of_day' column
df['time_of_day'] = df['trans_hour'].apply(map_time_of_day)

# Plot the histogram with the new 'time_of_day' column
plt.figure(figsize=(10, 5))
ax = sns.histplot(data=df, x='time_of_day', hue='isfraud', common_norm=False, stat='percent', multiple='dodge')
ax.set_xlabel('Time of Day')
ax.set_ylabel('Percentage')
plt.xticks(rotation=45)
plt.legend(title='Type', labels=['Fraud', 'Non-Fraud'])
plt.show()

df.time_of_day

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df):
    # Group the DataFrame by 'time_of_day' and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('time_of_day')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts for time of day
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized non-fraud-to-fraud ratio for each time category
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    # Calculate the normalized fraud-to-non-fraud ratio for each time category
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped

# Calculate the normalized non-fraud-to-fraud and fraud-to-non-fraud ratios for time of day
ratio_df = calculate_ratio(df)

# Plot both the non-fraud-to-fraud ratio and fraud-to-non-fraud ratio by time of day in a rotated graph
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(ratio_df['time_of_day'], ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(ratio_df['time_of_day'], -ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
plt.xlabel('Ratio')

plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.ylabel('Time of Day')
plt.title('Fraud and Non-fraud Ratios by Time of Day')

# Display the ratios on the right side of each bar
for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.yticks(rotation=45)  # Rotate y-axis labels
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
ax = sns.histplot(data= df, x='gender', hue='isfraud', stat='percent', common_norm = False, multiple = 'dodge')
ax.set_xlabel('Credit Card Holder Gender')
ax.set_ylabel('Percentage')
plt.legend(title = 'Type', labels= ['Fraud','Not-Fraud'])
plt.show()

In [ ]:
## Gender Variable##

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df):
    # Group the DataFrame by 'gender' and calculate the normalized counts of fraud and non-fraud transactions
    grouped = df.groupby('gender')['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts for gender
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized non-fraud-to-fraud ratio for each gender
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    # Calculate the normalized fraud-to-non-fraud ratio for each gender
    grouped['fraud_ratio'] = grouped[1] / grouped[0]

    return grouped

# Calculate the normalized non-fraud-to-fraud and fraud-to-non-fraud ratios for gender
ratio_df = calculate_ratio(df)

# Plot both the non-fraud-to-fraud ratio and fraud-to-non-fraud ratio by gender in opposite directions
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(ratio_df['gender'], ratio_df['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(ratio_df['gender'], -ratio_df['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio
plt.ylabel('Gender')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.xlabel('Ratio')
plt.title('Fraud and Non-fraud Ratios by Gender')
plt.legend()

# Display the ratio values on the bars
for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.show()


In [ ]:
# Calculate the value counts of each category
category_counts = df['category'].value_counts(normalize=True)

# Plot the count plot
plt.figure(figsize=(10, 5))
plot = sns.countplot(x='category', data=df, hue='isfraud')
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Transaction Count by Category')
plt.legend(title='Fraud', labels=['Non-Fraud', 'Fraud'])
plt.show()

In [ ]:
## Merchant Category Variable ##
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_ratio(df, column):
    # Group the DataFrame by the specified column and calculate normalized counts of fraud and non-fraud transactions
    grouped = df.groupby(column)['isfraud'].value_counts(normalize=True).unstack().reset_index()

    # Normalize both fraud and non-fraud counts
    grouped[0] /= grouped[0].sum()
    grouped[1] /= grouped[1].sum()

    # Calculate the normalized fraud-to-non-fraud ratio for each group
    grouped['fraud_ratio'] = grouped[1] / grouped[0]
    # Calculate the normalized non-fraud-to-fraud ratio for each group
    grouped['non_fraud_ratio'] = grouped[0] / grouped[1]

    return grouped

# Calculate the normalized fraud-to-non-fraud and non-fraud-to-fraud ratios for merchant categories
category_ratio_df = calculate_ratio(df, 'category')

# Sort the DataFrame by 'category' in ascending order
category_ratio_df_sorted = category_ratio_df.sort_values('category')

# Plot both normalized fraud and non-fraud ratios on the same bar chart
plt.figure(figsize=(10, 5))
fraud_bars = plt.barh(category_ratio_df_sorted['category'], category_ratio_df_sorted['fraud_ratio'], label='Fraud Ratio', color='blue')
non_fraud_bars = plt.barh(category_ratio_df_sorted['category'], -category_ratio_df_sorted['non_fraud_ratio'], label='Non-Fraud Ratio', alpha=0.7, color='orange')  # Use negative values for non-fraud ratio

for bar in fraud_bars:
    plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, f'{bar.get_width():.2f}', va='center', color='black', fontsize=10)

for bar in non_fraud_bars:
    plt.text(bar.get_width() - 0.01, bar.get_y() + bar.get_height()/2, f'{-bar.get_width():.2f}', va='center', ha='right', color='black', fontsize=10)

plt.xlabel('Ratio')
plt.ylabel('Category')
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: str(-x) if x < 0 else str(x)))
plt.title('Fraud and Non-Fraud Ratios by Category')
plt.legend(loc='lower right')  # Adjust the legend position
plt.tight_layout()
plt.show()


In [ ]:
# remove first name, last name, data of birth and transaction date/time
df.drop(['first','last','dob','transdatetranstime','transdate'], axis = 1, inplace= True)